# Représentation des nombres

In [ ]:
# importing libraries used in this book
import numpy as np
import matplotlib.pyplot as plt

## Représentation de nombres : Exemple $e$

L'expression suivante du nombre d'Euler $e$ est connue :
$$e = \lim_{n\to\infty}\left( 1 + \frac{1}{n} \right)^{n}.$$
 On s'attend donc à ce que $e_n = \left(1 + \frac{1}{n} \right)^{n}$ donne des approximations de plus en plus bonnes de $e$
 En arithmétique exacte, c'est effectivement le cas. Sur l'ordinateur, la suite \emph{calculée} $\hat e_n$ se comporte tout à fait différemment :


In [1]:
# Approximation of e, numpy package needed to include e
import numpy as np
print('10^i \t\t e_n \t\t\t e_n - e')
for i in range(1,16):
    n = 10.0 ** i; en = (1 + 1/n) ** n
    print('10^%2d \t %20.15f \t %20.15f' % (i,en,en-np.e))

10^i 		 e_n 			 e_n - e
10^ 1 	    2.593742460100002 	   -0.124539368359043
10^ 2 	    2.704813829421528 	   -0.013467999037517
10^ 3 	    2.716923932235594 	   -0.001357896223452
10^ 4 	    2.718145926824926 	   -0.000135901634120
10^ 5 	    2.718268237192297 	   -0.000013591266748
10^ 6 	    2.718280469095753 	   -0.000001359363292
10^ 7 	    2.718281694132082 	   -0.000000134326963
10^ 8 	    2.718281798347358 	   -0.000000030111687
10^ 9 	    2.718282052011560 	    0.000000223552515
10^10 	    2.718282053234788 	    0.000000224775742
10^11 	    2.718282053357110 	    0.000000224898065
10^12 	    2.718523496037238 	    0.000241667578193
10^13 	    2.716110034086901 	   -0.002171794372144
10^14 	    2.716110034087023 	   -0.002171794372022
10^15 	    3.035035206549262 	    0.316753378090217


## Représentation de nombres : Exemple $e^x$

La série de Taylor pour la fonction exponentielle converge pour tout $x \in \mathbb R$ :
$$e^x = \sum\limits_{k=0}^{\infty}\frac{x^{k}}{k!} = 
        1 + x + \frac{x^{2}}{2} + \frac{x^{3}}{6} + \frac{x^{4}}{24} + \dots.$$
L'ordinateur peut seulement calculer la série partielle
$$s_i(x) = \sum\limits_{k=0}^{i}\frac{x^{k}}{k!}.$$
Le reste de Taylor est donné par
$e^x - s_i(x)  = \frac{e^\xi x^{i+1}}{(i+1)!}
\text{ pour un } \xi \in \mathbb R \text{ avec }0<|\xi|<|x|$$

Si on choisi une tolerance $\mathsf{tol}>0$ et $i$ tel que
$|x|^{i+1} / (i+1)! \le \mathsf{tol} \cdot s_i(x)$, pour $x<0$ on obtient
$$|e^x - s_i(x)| \le   \frac{|x|^{i+1}}{(i+1)!}  \le \mathsf{tol} \cdot s_i(x) \approx \mathsf{tol} \cdot e^x.$$
Au même temps,  **l'erreur rélative** $|e^x - s_i(x)| / |e^x|$ est bornée par  $\mathsf{tol}$.


In [ ]:
def expeval(x, tol):
    #Approximation of e^x
    s = 1; k = 1
    term = 1
    while (abs(term)>tol*abs(s)):
        term = term * x / k
        s = s + term ; k = k + 1
    return s

In [ ]:
# reproduisez ici le tableau
# $x$ & Computed $\hat s_i(x)$ & $\exp(x)$ & $\frac{|\exp(x)-\hat s_i(x)|}{\exp(x)}$



## IEEE 754 in Python
En Python, toutes les opérations sur les nombres réels sont exécutées par défaut en double précision.
Les variables en simple précision sont générées par la commande ```numpy.float32()```.

In [ ]:
import sys
sys.float_info.min             # 2.2251e-308

In [ ]:
sys.float_info.max             # 1.7977e+308

In [ ]:
1 / 0                          # Divide by zero error

In [ ]:
3 * float('inf')               # np.inf

In [ ]:
-1 / 0                         # Divide by zero error

In [ ]:
0 / 0                          # Divide by zero error

In [ ]:
float('inf') - float('inf')    # nan

De manière assez surprenante, et non en accord avec le standard IEEE 754 https://wusun.name/blog/2017-12-18-python-zerodiv/, Python renvoit une erreur lorsqu'une division par zéro se produit, au lieu de retourner un $\infty$ signé. Pour éviter cela, on peut utiliser ``float64`` de ``numpy``. Par exemple, ``1/np.float64(0)`` renvoie ``inf`` (comme il se doit).


## Arrondis

### Sommes

Comme exemple, regardons ce qu'il se pase avec de ```float16``` et une somme de nombre de plus en plus petits.


In [ ]:
# see what happens with n = 2, 10, 100, 1000, 10000
n = 2 # COMPLETE HERE

# t and y are 64 bit float (double) numbers
t = np.sin(np.linspace(0,1,n))
y = np.linspace(0,1,n) 

y[1::2] = -1 * y[1::2] # odd numbers set to negatives

# reduced accuracy
x= np.float16(y) 
r= np.float16(t)

In [ ]:
# Use the sum function
print(sum(y*t))
print(sum(x*r))
print(np.float16(sum(x*r)))

In [ ]:
# Use the sum function, reverse order
# To reverse the order of an array, use y[::-1]

# COMPLETE HERE


In [ ]:
# loop for the sum from the first to last, in float16
s = np.float16(0);
for k in range(n) :
    s = s + x[k]*r[k]
print(s)

# loop for the sum from the first to last
s = np.float16(0);
for k in range(n) :
    s = s + x[n-k-1]*r[n-k-1]
print(s)


## Cancellation, exemple

Considérons
$$f(x) = \frac{1- \cos(x)}{x^2} \;, \quad x = b^{-k}, k=5,...,9.$$


In [ ]:
del x
def f (x):
    return (1 - np.cos(x))/ (x*x)

# Try with b = 2,3,4,10
b = 10

k=np.array(range(6,20))
x = 1/b**k

# Uncomment the lines below
# print('k \t', k)
# print('x=10^-k ',x)
# print('f(x) \t', f(x))

## Plot a semilog x graph with x and f(x)
# plt.semilogx(?, ?,'b*-')


# Labels
plt.xlabel('x'); plt.ylabel('f(x)'); plt.title('f(x) with respect to x')
plt.legend(['error'])
plt.show()


### Exemple, différences finies

Considérons $f(x) = e^x$, $x_0=0$ et 
$$f^\prime(x_0) = \lim_{h\to 0} \frac{f(x_0+h) - f(x_0)}{h}.$$

Attente : L'approximation de $f^\prime(x_0)$ par un quotient de différences finies tend à s'améliorer lorsque $h$ s'approche de $0$. 

In [ ]:

def f(x) :
    return np.exp(x)
def df(x) :
    return np.exp(x)
x0 = 0

# prenez h = 10{-k} avec k=0,1,...,16
k = np.array(range(0,17))
h = 1/10**k

# calculez la difference finie (f(x+h) - f(x))/h pour tout les h
fprime = (f(h) - f(0))/h

# calculez l'erreur avec la solution exacte errH = |df(x0) - ..|
# COMPLETEZ ICI

# dessinez l'erreur en log-log 

# Labels
plt.xlabel('h'); plt.ylabel('ErrH'); plt.title('Error in finite difference')
plt.legend(['error'])
plt.show()


### Exemple, le nombre $e$

$$e = \lim_{n\to \infty} \big( 1 + \frac1n)^n$$

Attente : plus $n$ est grand, plus $e_n = \big( 1 + \frac1n)^n$ s'approche de  $e$ 


In [ ]:
import matplotlib.pyplot as plt

def limN(n) :
    return (1+1/n)**n

# prenez n = 10{k} avec k=0,1,...,16
k = np.array(range(0,17))
n = 10**k

# calculez (1+1/n)**n
approxE = limN(n)

# calculez l'erreur avec la solution exacte errN = |e - ..|
# COPLETEZ ICI    


# dessinez l'erreur en log-log 

# Labels
plt.xlabel('n'); plt.ylabel('ErrN'); plt.title('Error in finite difference')
plt.legend(['error'])
plt.show()
